In [102]:
import os
import pandas as pd
import re

def masterfile():
    # Paths
    folderPath = "FOLDER"  # Update this
    masterPath = "Master datasheet.xlsx"  # Update this
    outputPath = "master/master.xlsx"  # Update this

    def generateCol(n):
        """ Generate Excel-style column names (A, B, C... AA, AB, etc.). """
        names = []
        for i in range(n):
            col_name = ''
            while i >= 0:
                col_name = chr(i % 26 + 65) + col_name
                i = i // 26 - 1
            names.append(col_name)
        return names

    # Read the master template
    master = pd.read_excel(masterPath)
    masterColnames = master.columns  # Preserve original column names
    n = len(master.columns)
    Colnames = generateCol(n)
    master.columns = Colnames  # Use temporary column names for processing

    # Loop through all folders in the directory
    for foldername in os.listdir(folderPath):
        paths = os.path.join(folderPath, foldername)
        if not os.path.isdir(paths):  # Skip files, only process directories
            continue
        
        for filename in os.listdir(paths):
            filePath = os.path.join(paths, filename)

            try:
                # Read the column mapping sheet (handle missing sheets)
                try:
                    Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
                except ValueError:
                    print(f"Warning: Sheet '{foldername}' not found in master template. Skipping.")
                    continue

                if not {'From', 'To'}.issubset(Col_directions.columns):
                    print(f"Error: Mapping sheet {foldername} must contain 'From' and 'To' columns.")
                    continue

                # Read the file to process
                fileRead = pd.read_excel(filePath)
                temp_data = []  # Temporary storage for processed rows

                # Breaking measurement columns into M1, M2, M3
                if 'Measurement' in fileRead.columns:
                    print(fileRead.columns)
                    fileRead[['m1', 'm2', 'm3']] = fileRead['Measurement'].str.split(' x ', expand=True)
                    fileRead[['m1', 'm2', 'm3']] = fileRead[['m1', 'm2', 'm3']].apply(pd.to_numeric, errors='coerce')
                    # print(fileRead[['m1', 'm2', 'm3']])

                def extract_dimensions(measurement):
                    match = re.match(r"(\d+\.\d+)\s*-\s*(\d+\.\d+)\s*\*\s*(\d+\.\d+)", str(measurement))
                    return match.groups() if match else (None, None, None)
           
                if 'Diameter' in fileRead.columns:
                    fileRead[['m1', 'm2', 'm3']] = fileRead['Diameter'].apply(lambda x: pd.Series(extract_dimensions(x)))
                    # print(fileRead[['m1', 'm2', 'm3']])
                # Assign dynamic column names to the current file
                n = len(fileRead.columns)
                Colnames = generateCol(n)
                fileRead.columns = Colnames

                # Process rows based on column mapping
                for _, row in fileRead.iterrows():
                    # if pd.isna(row['A']):  # Stop processing if empty row
                    #     break

                    new_row = {'A': filename}  # Track file name for reference

                    for _, movement_row in Col_directions.iterrows():
                        from_col = movement_row['From']
                        to_col = movement_row['To']

                        if from_col in row.index and to_col in master.columns:
                            new_row[to_col] = row[from_col]

                    temp_data.append(new_row)  # Add processed row

                # Append processed data to master
                temp_df = pd.DataFrame(temp_data)
                master = pd.concat([master, temp_df], ignore_index=True)

            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                continue
            
    # Function to calculate 'R' column
    def calculate_R(row):
        if 'P' in row and 'Q' in row and pd.notna(row['P']) and pd.notna(row['Q']):
            return ((100 - row['Q']) / 100) * row['P'] if row['Q'] > 0 else ((100 + row['Q']) / 100) * row['P']
        return None

    # Function to calculate 'S' column
    def calculate_S(row):
        if 'R' in row and 'C' in row and pd.notna(row['R']) and pd.notna(row['C']):
            return round(row['R'] * row['C'])  # Perform calculation
        return None

    # Apply calculations
    if 'P' in master.columns and 'Q' in master.columns:
        master['R'] = master.apply(calculate_R, axis=1)
    if 'R' in master.columns and 'C' in master.columns:
        master['S'] = master.apply(calculate_S, axis=1)

    # Clean column 'I'
    if 'I' in master.columns:
        master['I'] = master['I'].apply(lambda x: None if pd.isna(x) or str(x).strip() == '' else x)

    # Restore original column names
    master.columns = masterColnames

    # Save the final master file
    os.makedirs(os.path.dirname(outputPath), exist_ok=True)  # Ensure directory exists
    master.to_excel(outputPath, index=False)
    print(f"Master file updated and saved at {outputPath}.")



In [103]:
import os
import pandas as pd
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils import column_index_from_string

def hyperlink():
    folderPath = "FOLDER"
    masterPath = "Master datasheet.xlsx"
    outputPath = "master/master.xlsx"
    targetCol = column_index_from_string('X')  # Column A for data
    # Column B for filenames
    ReferenceCol = column_index_from_string('AD')
    fileCol = column_index_from_string('AC')
    rowCount = 2

    if os.path.exists("master\\master.xlsx"):
        target = load_workbook("master\\master.xlsx")
        targetSheet = target.active
     # Add headers

    for foldername in os.listdir(folderPath):
        paths = os.path.join(folderPath, foldername)

        for filename in os.listdir(paths):
            try:
                filePath = os.path.join(paths, filename)
                Col_directions = pd.read_excel(masterPath, sheet_name=foldername)
                source = load_workbook(filePath)
                sourceSheet = source.active

                sourceCol = column_index_from_string(Col_directions['Hfrom'][0])
                RefCol = column_index_from_string(Col_directions['StkRf'][0])

                for count, row in enumerate(sourceSheet.iter_rows(min_row=2), start=2):
                    cell = row[sourceCol - 1]
                    RefCell = row[RefCol-1]
                    targetCell = targetSheet.cell(row=rowCount, column=targetCol)
                    targetCell.value = cell.value
                    
                    

                    # Store filename in column B
                    

                    if cell.hyperlink:
                        targetCell.hyperlink = cell.hyperlink
                        targetCell.style = "Hyperlink"

                        rowCount += 1

                print(f"Processed {filename} successfully")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    target.save(outputPath)
    print("Done")






In [ ]:
import pandas as pd
import psycopg2
from openpyxl import load_workbook

def database_transfer():
    conn = psycopg2.connect(
        user="postgres",
        host="localhost",
        database="Master_Compilation",
        password="namanjain",
        port=5432,
    )
    db = conn.cursor()

    # Create table if not exists
    create_table_query = """
    CREATE TABLE IF NOT EXISTS diamonds (
        date DATE NOT NULL,
        stock_ref TEXT NOT NULL,
        company TEXT,
        shape TEXT,
        size NUMERIC(5,2),
        colour TEXT,
        clarity TEXT,
        cut TEXT,
        polish TEXT,
        symmetry TEXT,
        fluorescence TEXT,
        ratio NUMERIC(6,3),
        M1 NUMERIC(6,2),
        M2 NUMERIC(6,2),
        M3 NUMERIC(6,2),
        depth NUMERIC(5,2),
        table_percent NUMERIC(5,2),
        rap_rate NUMERIC(10,2),
        discount NUMERIC(5,2),
        ppc NUMERIC(10,2),
        total NUMERIC(10,2),
        total_in_rs NUMERIC(10,2),
        report_no TEXT,
        cert_link TEXT,
        video_link TEXT CHECK (video_link ~* '^https?://.+'),
        bin_c TEXT,
        bin_s TEXT,
        comment TEXT,
        comment_2 TEXT,
        PRIMARY KEY (date, stock_ref)
    );
    """
    db.execute(create_table_query)
    conn.commit()

    # Read Excel file
    file = "master/master.xlsx"
    wb = load_workbook(file, data_only=True)
    ws = wb.active
    df = pd.read_excel(file, engine="openpyxl")

    # Extract hyperlinks for the "Video link" column
    video_links = []
    for row in ws.iter_rows(
        min_row=2, max_row=ws.max_row, min_col=df.columns.get_loc("Video link") + 1, max_col=df.columns.get_loc("Video link") + 1
    ):
        cell = row[0]
        video_links.append(cell.hyperlink.target if cell.hyperlink else None)
    
    df["Video link"] = video_links

    # Convert numeric columns
    numeric_columns = ["Size", "Ratio", "M1", "M2", "M3", "Depth", "Table", "Rap rate", "Discount", "PPC", "Total"]
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Insert data
    insert_query = """
     insert into diamonds (Company,Shape,Size,ColourCclarity,Cut,Polish,Symmetry,
            Flour,Ratio,M1,M2,M3,Depth,Tables,rap_rate,discount,ppc,total,total_in_rs,report_no,
            stockref,cert_link,video_link,binC,binS,comment,comment2) Values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ON CONFLICT (date, stock_ref) DO NOTHING;
    """
    print(list(df.columns))

    for _, row in df.iterrows():
        # print(row.values)
        db.execute(insert_query, row)

    conn.commit()
    db.close()
    conn.close()
    print("Successfully added!")

database_transfer()

In [ ]:
import pandas as pd
import psycopg2
from openpyxl import load_workbook

def database_transfer():
    conn = psycopg2.connect(
        user= "postgres",
        host= "localhost",
        database= "Master_Compilation",
        password="namanjain",
        port= 5432,
    )
    db = conn.cursor()
    file = "master\master.xlsx"
    wb = load_workbook(file, data_only=True)
    ws = wb.active
    df = pd.read_excel(file,engine="openpyxl")
    columns = list(df.columns)
    columns.append('Date')
    vedio_links=[]
    for row in ws.iter_rows(min_row=2,max_row=ws.max_row,min_col=df.columns.get_loc("Video link")+1,max_col=df.columns.get_loc("Video link")+1):
        cell = row[0]
        if cell.hyperlink:
            vedio_links.append(cell.hyperlink.target)
        else:
            vedio_links.append(None)
    df['Video link'] =  vedio_links
    numeric_columns = ["Size", "Ratio", "M1", "M2", "M3", "Depth", "Table",
                   "Rap rate", "Discount", "PPC", "Total"]
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col],errors='coerce')
    display(df)
    for index, row in df.iterrows():
        db.execute(''' insert into diamonds (company,shape,size,colour,clarity,cut,polish,symmetry,
                flour,ratio,m1,m2,m3,depth,tables,rap_rate,discount,ppc,total,total_in_rs,report_no,
                stockref,cert_link,video_link,binC,binS,comment,comment2) Values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''',
                tuple(row))
    conn.commit()
    db.close()
    conn.close()
    print("successfully added")
    
        
database_transfer()

In [104]:
import threading

t1= threading.Thread(target=masterfile)
t2 = threading.Thread(target=hyperlink)

t1.start()
t1.join()
t2.start()
t2.join()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13848\1775762361.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  master = pd.concat([master, temp_df], ignore_index=True)


Index(['SrNo', 'Location', 'Image', 'View360', 'LAB', 'DNA', 'Verify', 'Refno',
       'Group', 'Shape', 'Carat', 'Threads', 'Rapaport', 'Price/CTS RS',
       'TotalValue RS', 'OffPer', 'RapOff5Cts', 'BidType', 'BID(%)',
       'BID Price', 'BIDValidTill', 'Color', 'CS', 'Clarity', 'Cut', 'Polish',
       'Symm', 'Fl', 'FC', 'Measurement', 'TD%', 'Tab%', 'NOBGM', 'Grd%',
       'CrAngPer', 'HA', 'Ratio', 'TI', 'BIS', 'BIC', 'TOI', 'EyeClean',
       'Luster', 'CrownHeight', 'PavalionHeight', 'PavalionAngle', 'CU',
       'OPPV', 'OPCR', 'OPTA', 'EFCR', 'EFPV', 'Natural-PV', 'Natural-CR',
       'Natural-GRD', 'INatural-PV', 'INatural-CR', 'INatural-GRD', 'Graining',
       'SurfaceGraining', 'GirdleDesc', 'GirdleMinMax%', 'GC', 'StarLength',
       'LowerHalf', 'Cert. Type', 'Cert.No', 'Cert.Date', 'MultiGroup',
       'Type2', 'FM', 'DOR Code', 'SealedCode', 'MineName', 'QC', 'INS',
       'RoughOrigin', 'DOR No', 'LabControlNo', 'KeytoSymbols',
       'ReportComments', 'RPComment', 

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_13848\1775762361.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  master = pd.concat([master, temp_df], ignore_index=True)


Index(['Sr.No', 'Packet No', 'Status / Location', 'Cert', 'Shape', 'Carats',
       'Price/Cts INR', 'Disc%', 'Tbl Incl', 'Measurement', 'L:W', 'Grd %',
       'Incl Ptrn', 'Int Grn', 'Int Grn Typ', 'Tbl Opn', 'Crn Opn', 'Pav Opn',
       'Grd Crn Opn', 'LP', 'Blk Incl', 'Table Black', 'Clarity', 'Color',
       'Type2 Certi', 'Color Shade', 'Cut', 'Pol', 'Sym', 'Fluro', 'Fl. Col',
       'FE', 'Luster', 'Naked Eye', 'Depth %', 'Table %', 'Polished',
       'Front Hand', 'Back Hand', ' Tweezer', 'Light Video', 'Dark Video',
       'Video with Details', 'MP4 Video', 'Plotting', 'Fluorescence',
       'Journey', 'Consumer Video', 'ASET', 'Hearts & Arrows', 'Key To Symbol',
       'Additional Comments', 'Disc Price', 'Disc Total', 'Total Amt'],
      dtype='object')
Master file updated and saved at master/master.xlsx.
Processed DDPLStock-25-Feb-2025 17_49_28.xlsx successfully
Processed Finestar.xlsx successfully
Processed Jbbros rounds.xlsx successfully
Processed Jodhani-Brothers-Stock-25